# MAGICIAN: Mining and gAmblingG servIces sCrapIng and ANalysis


## Setting up the environment for the project


In [ ]:
# Required libraris 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import time



### Configuration for the scraping process


In [ ]:
BASE_URL = "https://www.walletexplorer.com"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/114.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    }

### Check the robot.txt file


In [ ]:
import urllib.robotparser

# Parse the robots.txt file 
rp = urllib.robotparser.RobotFileParser(BASE_URL + "/robots.txt")
rp.read()

if not rp.mtime() or not rp.can_fetch("*", BASE_URL + "/robots.txt"):
   print("robots.txt could not be read or is not present.")


# Extracting DeepBit.net and DiceOnCrack.com wallet addresses

I create a small pipeline to scrape the walletexplorer website and then extract the wallet addresses from the pages of the two websites by using two functions.
The idea is to firstly scrape the main page of the walletexplorer website and then for each service, scrape the wallet addresses page and extract the addresses from the table. For each request to the walletexplorer website, before proceding I wait 5 seconds to avoid overwhelming the server otherwise I would get ban temporarily from the website.


In [ ]:
def get_walletexplorer_page():
    """ 
    Retrieve the main page of WalletExplorer

    This function sends a GET request to the WalletExplorer main page using the specified headers and before sending the request it waits 5 seconds before making the request to avoid overwhelming the server.
    Returns the response object if successful, otherwise returns None.
    """
    try:
        time.sleep(5)  # Pause for 5 seconds to avoid overwhelming the server
        print("Accessing WalletExplorer main page...")
        main_walletexplore_page = requests.get(BASE_URL, headers=HEADERS)
        main_walletexplore_page.raise_for_status()
    except requests.exceptions.RequestException as e:
        print("Error while accessing WalletExplorer:", e)
        return None
    return main_walletexplore_page

def scraping_wallet_address(html_page,service_name):
    """
    Scrape the wallet addresses of a specific service from WalletExplorer.

    This function takes the HTML page of WalletExplorer and the name of the service as input.
    Firstly, it accesses the page of the service by searching for its name, secondly, it navigates to the wallet addresses page and in the end, it extracts the wallet addresses from the table on that page.
    Returns a pandas Series of wallet addresses (hashes) or an empty Series if none are found.

    Parameters:
    - html_page: str, the HTML page of WalletExplorer.
    - service_name: str, the name of the service to search for.
    Returns:
    - pd.Series: a pandas Series containing the wallet addresses (hashes) of the service, or an empty Series if none are found.
    """

    # Open search page regarding 'service_name' and open the wallet addresses page
    try: 
        time.sleep(5)  # Pause for 5 seconds to avoid overwhelming the server
        print(f"Accessing the search page for '{service_name}'...")

        search_page = requests.get(html_page, headers=HEADERS, params={'wallet' :service_name}, timeout=60)
        search_page.raise_for_status()
    except requests.exceptions.RequestException as e:
        print("Error while accessing the search page:", e)
        return None


    # Scrape the search results and extract the wallet addresses of 'service_name'
    soup = BeautifulSoup(search_page.text, 'html.parser')

    # Find the url of the wallet addresses page
    span = soup.find('span', {'class': 'showother'})

    wallet_link = span.find('a').get('href')
    wallets_url = BASE_URL + wallet_link # create the full URL for the wallet addresses page

    # Access the wallet addresses page using the URL found in the search results
    try:
        time.sleep(5)  # Pause for 5 seconds to avoid overwhelming the server        
        wallet_addr_page = requests.get(wallets_url, headers=HEADERS)
        wallet_addr_page.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error while accessing the wallets page: {e}")
        return None     

    # Scrape the wallet addresses page extracting the information from the table
    soup = BeautifulSoup(wallet_addr_page.text, 'html.parser')

    # Save the wallet address of 'service_name'
    tmp = []

    # Find the table containing the wallet addresses
    wallet_table = soup.find('table')

    for row in wallet_table.find_all('tr'):
        col = row.find('td')
        if col and col.find('a', href=True):
            addr = col.find('a')
            tmp.append(addr.text.strip())
    
    return pd.Series(tmp, name='hash') if tmp else pd.Series([], name='hash')

In [ ]:
deepbit_service = "DeepBit.net"
diceoncrack_service = "DiceOnCrack.com"

# Open the main page of WalletExplorer
main_walletexplore_page = get_walletexplorer_page()
# Check if the main page was retrieved successfully
if main_walletexplore_page is None:
    print("Failed to retrieve the main WalletExplorer page.")
else:
    # Search the form in the page
    soup = BeautifulSoup(main_walletexplore_page.text, 'html.parser')
    search_form = soup.find('form', {'class':'main'})
    action_form = search_form.get('action')

    # Create the target URL for the search form
    if action_form.startswith('/'):
        target_url = BASE_URL + action_form
    else:
        target_url = BASE_URL+ '/'+action_form


    # Get the wallet addresses for DeepBit.net
    print(f"Searching for wallet addresses of {deepbit_service}...")
    deepbit_wallet_addresses = scraping_wallet_address(target_url, deepbit_service)
    if deepbit_wallet_addresses is None:
        print(f"Failed to retrieve wallet addresses for {deepbit_service}.\n")
    else :
        print(f"Wallet addresses for {deepbit_service} have been successfully retrieved.\n")
    # Get the wallet addresses for DiceOnCrack.com
    print(f"Searching for wallet addresses of {diceoncrack_service}...")
    diceoncrack_wallet_addresses = scraping_wallet_address(target_url, diceoncrack_service)
    if diceoncrack_wallet_addresses is None:
        print(f"Failed to retrieve wallet addresses for {diceoncrack_service}.\n")
    else :
        print(f"Wallet addresses for {diceoncrack_service} have been successfully retrieved.\n")
#Print the results
print(f"DeepBit.net wallet addresses: {deepbit_wallet_addresses}")
print(f"DiceOnCrack.com wallet addresses: {diceoncrack_wallet_addresses}")


#### Load datasets for the next analysis regarding the service DeepBit.net and DiceOnCrack.com

Given that the great size of the datasets, I decided to read the csv files only one time in order to avoid to read them multiple times during the analysis. I used Pyarrow engine to read the csv files due to its speed and efficiency in handling large datasets.


In [ ]:
# Load datasets
transactions = pd.read_csv('transactions.csv', engine='pyarrow')
transactions.columns = ['timestamp', 'blockId', 'txId', 'isCoinbase', 'fee']

outputs = pd.read_csv('outputs.csv', engine='pyarrow')
outputs.columns = ['txId', 'position', 'addressId', 'amount', 'scripttype']

inputs = pd.read_csv('inputs.csv', engine='pyarrow')
inputs.columns = ['txId', 'prevTxId', 'prevTxpos']

mapping = pd.read_csv('mapping.csv', engine='pyarrow')
mapping.columns = ['hash', 'addressId']

# Deepbit.net's mining pool analysis

Subsequently, I will analyze the dataset of the Deepbit.net mining pool. The analysis will focus on the distribution of the mined blocks, the distribution of the received fees and the computation of the UTXO (Unspent Transaction Output) regarding the service Deepbit.net


In [ ]:
def map_wallet_addresses(wallet_addresses,mapping_dt):
    """
    Map the wallet addresses to their corresponding addressId in the mapping dataset.
    Returns a Series of addressId found or an empty Series if none found.

    Parameters:
    - wallet_addresses: pd.Series, a Series of wallet addresses (hashes) to be mapped.
    - mapping_dt: pd.DataFrame, a DataFrame containing the mapping of wallet addresses to addressId.
    Returns:
    - pd.Series: a pandas Series containing the addressId corresponding to the wallet addresses, or an empty Series if none are found.
    """
    mapped_addresses = mapping_dt.merge(wallet_addresses,on ='hash')['addressId']
    if mapped_addresses.empty:
        return pd.Series([], name='addressId')
    
    return mapped_addresses 

In [ ]:
#1) Identify the mapping between wallet addresses on the dataset
deepbit_mapped_addresses = map_wallet_addresses(deepbit_wallet_addresses,mapping)

print(deepbit_mapped_addresses.head())


## 1) Deepbit.net's mined block distribution


Firslty, it needs to extract the transactions of DeepBit.net from the datasets, which are the transactions that have at least one input or output address of DeepBit.net.


In [ ]:
def find_wallet_transactions(wallet_mapped_addresses, transactions, inputs, outputs):
    """
    Finds all transactions related to the specified service addresses.
    This function takes the mapped addresses of a service and the transactions, inputs, and outputs datasets and the retrieves all transactions that have at least one output address of the service.
    It returns a DataFrame containing all transaction details related to the service.

    Parameters:
    - wallet_mapped_addresses: pd.Series, a Series of addressId corresponding to the wallet
    addresses of the service.
    - transactions: pd.DataFrame, a DataFrame containing transaction details.
    - inputs: pd.DataFrame, a DataFrame containing input details of transactions.
    - outputs: pd.DataFrame, a DataFrame containing output details of transactions.
    Returns:
    - pd.DataFrame: a DataFrame containing all transaction details related to the service.
    """

    # Find all outputs information (address, amount, ...) regardin the service
    diceoncrack_output_transactions = outputs.merge(wallet_mapped_addresses,on='addressId') 
    
    # Find all inputs information (address, amount, ...) regarding the service and rename the columns to prepare for the merge between inputs and outputs dataframes
    renamed_outputs= diceoncrack_output_transactions.rename(columns={'txId': 'prevTxId', 'position': 'prevTxpos'})
    wallet_input_transactions = inputs.merge(renamed_outputs,on=['prevTxId', 'prevTxpos'])

    # Make the union of all transactions that have at least one output address of the service
    df_union = pd.concat([diceoncrack_output_transactions['txId'], wallet_input_transactions['txId']], axis=0,ignore_index=True).drop_duplicates().reset_index(drop=True)

    # Merge the union of transactions with the transactions dataset to get all transaction details
    wallet_transactions = transactions.merge(df_union,on='txId') 

    return wallet_transactions


# Find the transactions related to Deepbit.net
deepbit_transactions = find_wallet_transactions(deepbit_mapped_addresses, transactions, inputs, outputs)

### Find the distribution of the mined blocks by DeepBit.net in the dataset


In [ ]:
# Find the transaction patterns for the wallet addresses

# 1. Select all coinbase transactions (mining rewards) from the transactions dataset and merge them with the outputs dataset in order to get their output information 
coinbase_transactions = transactions[transactions['isCoinbase'] == 1]['txId']
merged_outputs_first_transact= outputs.merge(coinbase_transactions, on='txId')

# 2. Find all outputs where Deepbit.net addresses appear (as outputs)
deepbit_asoutput_transactions = outputs.merge(deepbit_mapped_addresses, on='addressId')
tmp = deepbit_asoutput_transactions.merge(deepbit_transactions, on='txId')
inputs_second_transaction = tmp.merge(inputs, on='txId')

# Merge to find the base pattern: transactions where Deepbit.net spends outputs from mined blocks
merged_outputs_first_transact.rename(columns={'txId': 'prevTxId', 'position': 'prevTxpos'}, inplace=True)
deepbit_mined_block_transact = inputs_second_transaction.merge(merged_outputs_first_transact, on=['prevTxId', 'prevTxpos']).drop_duplicates()[['txId', 'prevTxId', 'prevTxpos','timestamp']]

# Lets prepare the data for Deepbit.net block distribution
deepbit_block_distribution = deepbit_mined_block_transact.copy()
deepbit_block_distribution['timestamp'] = pd.to_datetime(deepbit_block_distribution['timestamp'], unit='s')
deepbit_block_distribution.set_index('timestamp', inplace=True)

# 1. Daily count
daily = deepbit_block_distribution.resample('D').size()
# 2. Weekly count
weekly = deepbit_block_distribution.resample('W').size()
# 3. Monthly count
monthly = deepbit_block_distribution.resample('ME').size()

# Visualization
fig, axs = plt.subplots(3, 2, figsize=(18, 12))
fig.suptitle('Distribution of blocks mined by Deepbit.net', fontsize=16)

# Line plots (left)
axs[0, 0].plot(daily.index, daily.values, label='Daily', color='blue')
axs[0, 0].set_title('Mined blocks - Daily')
axs[0, 0].set_ylabel('Blocks')
axs[0, 0].grid(True, alpha=0.3)

axs[1, 0].plot(weekly.index, weekly.values, label='Weekly', color='green')
axs[1, 0].set_title('Mined blocks - Weekly')
axs[1, 0].set_ylabel('Blocks')
axs[1, 0].grid(True, alpha=0.3)

axs[2, 0].plot(monthly.index, monthly.values, label='Monthly', color='orange')
axs[2, 0].set_title('Mined blocks - Monthly')
axs[2, 0].set_ylabel('Blocks')
axs[2, 0].set_xlabel('Date')
axs[2, 0].grid(True, alpha=0.3)

# Bar plots (right)
axs[0, 1].bar(daily.index, daily.values, color='blue')
axs[0, 1].set_title('Bar plot - Daily')
axs[0, 1].set_ylabel('Blocks')
axs[0, 1].grid(True, alpha=0.3)

axs[1, 1].bar(weekly.index, weekly.values, color='green', width=5)
axs[1, 1].set_title('Bar plot - Weekly')
axs[1, 1].set_ylabel('Blocks')
axs[1, 1].grid(True, alpha=0.3)

axs[2, 1].bar(monthly.index, monthly.values, color='orange', width=20)
axs[2, 1].set_title('Bar plot - Monthly')
axs[2, 1].set_ylabel('Blocks')
axs[2, 1].set_xlabel('Date')
axs[2, 1].grid(True, alpha=0.3)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


## 2) DeepBit.net's fee distribution

Find how fee transactions are distributed to the DeepBit.net's mining pool


In [ ]:
# Identify coinbase transactions information
renamed_outputs = outputs.copy()
renamed_outputs.rename(columns={'txId': 'prevTxId', 'position': 'prevTxpos'}, inplace=True)
fee_info= renamed_outputs.merge(deepbit_mined_block_transact, on=['prevTxId','prevTxpos']).merge(transactions[['txId', 'blockId', 'fee']], on='txId')

# Convert timestamp to datetime and create a time series
fee_info['timestamp'] = pd.to_datetime(fee_info['timestamp'], unit='s')
fee_info.set_index('timestamp', inplace=True)

# Define the blockId of the first Bitcoin halving event.
first_btc_halving_blockId = 210000
# Convert the output amount from satoshis to BTC
fee_info['amount'] = fee_info['amount'] / 1e8  

# Calculate the expected block reward (50 BTC before halving, 25 BTC after) and compute the actual fee in BTC for each mined block transaction
fee_subtract = (fee_info['blockId'] < first_btc_halving_blockId).map({True: 50, False: 25})
fee_info['fee_btc'] = (fee_info['amount']+fee_info['fee'] - fee_subtract).round(8)


# Aggregations
daily_fee   = fee_info['fee_btc'].resample('D').sum()
weekly_fee  = fee_info['fee_btc'].resample('W').sum()
monthly_fee = fee_info['fee_btc'].resample('ME').sum()

# Visualization
fig, axs = plt.subplots(3, 2, figsize=(18, 12))
fig.suptitle('Distribution of fees from Deepbit.net', fontsize=16)

# Daily bar plot
axs[0,0].bar(daily_fee.index, daily_fee.values)
axs[0,0].set_title('Daily BTC fees')
axs[0,0].set_xlabel('Date')
axs[0,0].set_ylabel('Total fee (BTC)')
axs[0,0].grid(True, alpha=0.3)

# Weekly bar plot
axs[1,0].bar(weekly_fee.index, weekly_fee.values, color='green', width=5)
axs[1,0].set_title('Weekly BTC fees')
axs[1,0].set_xlabel('Week (end of week)')
axs[1,0].set_ylabel('Total fee (BTC)')
axs[1,0].grid(True, alpha=0.3)

# Monthly bar plot
axs[2,0].bar(monthly_fee.index, monthly_fee.values, color='orange', width=20)
axs[2,0].set_title('Monthly BTC fees')
axs[2,0].set_xlabel('Month')
axs[2,0].set_ylabel('Total fee (BTC)')
axs[2,0].grid(True, alpha=0.3)

# Daily bar plot (log scale)
axs[0,1].bar(daily_fee.index, daily_fee.values)
axs[0,1].set_yscale('log')
axs[0,1].set_title('Daily BTC fees (log scale)')
axs[0,1].set_xlabel('Date')
axs[0,1].set_ylabel('Total fee (BTC)')
axs[0,1].grid(True, alpha=0.3)

# Weekly bar plot (log scale)
axs[1,1].bar(weekly_fee.index, weekly_fee.values, color='green', width=5)
axs[1,1].set_yscale('log')
axs[1,1].set_title('Weekly BTC fees (log scale)')
axs[1,1].set_xlabel('Week (end of week)')
axs[1,1].set_ylabel('Total fee (BTC)')
axs[1,1].grid(True, alpha=0.3)

# Monthly bar plot (log scale)
axs[2,1].bar(monthly_fee.index, monthly_fee.values, color='orange', width=20)
axs[2,1].set_yscale('log')
axs[2,1].set_title('Monthly BTC fees (log scale)')
axs[2,1].set_xlabel('Month')
axs[2,1].set_ylabel('Total fee (BTC)')
axs[2,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 3) DeepBit.net's UTXO distribution

Compute the UTXO (Unspent Transaction Output) distribution of the DeepBit.net for each month in the dataset


In [ ]:
# UTXO are related to the unspent outputs of a transaction.

# Merge DeepBit.net related transactions with outputs to get all outputs with their timestamps
utxo_transactions = outputs.copy().merge(deepbit_transactions[['txId', 'timestamp']], on='txId')
utxo_transactions.rename(columns={'txId': 'prevTxId', 'position': 'prevTxpos'}, inplace=True)

# Merge with inputs to check which outputs have been spent
spent_utxo = utxo_transactions.merge(inputs[['prevTxId', 'prevTxpos']],on=['prevTxId', 'prevTxpos'], how='left', indicator='is_spent')

# Identify unspent outputs (UTXO)
unspent_utxo = spent_utxo[spent_utxo['is_spent'] == 'left_only'].drop(columns=['is_spent'])

# Convert the amount from satoshis to BTC and the timestamp to datetime.
unspent_utxo['amount'] = unspent_utxo['amount'] / 1e8 
unspent_utxo['timestamp'] = pd.to_datetime(unspent_utxo['timestamp'], unit='s')

# Create a range of month-end dates covering the period of the UTXOs.
all_dates = pd.date_range(
    start=unspent_utxo['timestamp'].min().to_period('M').to_timestamp(),
    end=unspent_utxo['timestamp'].max(),
    freq='ME'
)
# Adjust the end of the month to include the last second of the month (for example, 2011-11-31 23:59:59)
all_dates = all_dates.floor('D') + pd.Timedelta(days=1) - pd.Timedelta(seconds=1)

# Calculate monthly UTXO
# For each month, calculate the sum and count of UTXOs available before the end of that month.
monthly_utxo =[]
for month_end in all_dates:
    before_month_end_utxo = unspent_utxo[unspent_utxo['timestamp'] < month_end]
    sum_utxo_btc = before_month_end_utxo['amount'].sum()
    utxo_count = before_month_end_utxo['amount'].count()
    monthly_utxo.append([month_end, sum_utxo_btc, utxo_count])

monthly_utxo_df = pd.DataFrame(monthly_utxo, columns=['month', 'utxo_btc', 'utxo_count'])

# Visualize the monthly UTXO using bar plots
fig, axs = plt.subplots(2, 1, figsize=(10, 10))
fig.suptitle('Monthly distribution of DeepBit.net UTXOs', fontsize=16)

# Bar plot UTXO BTC
axs[0].bar(monthly_utxo_df['month'], monthly_utxo_df['utxo_btc'], color='blue', width=20, align='center')
axs[0].set_ylabel('UTXO (BTC)')
axs[0].set_title('Monthly sum of UTXOs (BTC)')
axs[0].grid(True, alpha=0.3)

# Bar plot UTXO Count
axs[1].bar(monthly_utxo_df['month'], monthly_utxo_df['utxo_count'], color='orange', width=20, align='center')
axs[1].set_ylabel('Number of UTXOs')
axs[1].set_title('Monthly number of UTXOs')
axs[1].set_xlabel('Month')
axs[1].grid(True, alpha=0.3)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


## Deepbit.net graphs parts


Preparing the data for the graphs of DeepBit.net


In [ ]:
import networkx as nx

# Find the transaction with txId 1883820
transaction = transactions[transactions['txId'] == 1883820].merge(outputs, on='txId')

### 1)


In [ ]:
deepbit_graph = nx.MultiDiGraph()

# Initially add the unique output addresses to the graph

# Aggiungi nodo con tutti gli attributi della riga
deepbit_graph.add_node(transaction.iloc[0]['txId'])


output_info = outputs[outputs['txId'] >= transaction.iloc[0]['txId']].copy()
inputs_info = inputs[inputs['txId'] >= transaction.iloc[0]['txId']].copy()


current_transaction = transaction

OthersDeepbit =set() # outputs not belonging to Deepbit.net

while True:
    # Find the inputs for the current transaction
    print(f"Processing transaction {current_transaction.iloc[0]['txId']}...")
    outs = output_info.merge(current_transaction['txId'], on='txId')
    
    # check if there is a Deepbit.net address
    check_deepbit_addr= outs.merge(deepbit_mapped_addresses, on='addressId', how='left', indicator='is_deepbit').rename(columns={'txId': 'prevTxId', 'position': 'prevTxpos'})
    deepbit_addrs = check_deepbit_addr[check_deepbit_addr['is_deepbit'] == 'both']
    other_addrs = check_deepbit_addr[check_deepbit_addr['is_deepbit'] == 'left_only']

    if len(deepbit_addrs) !=1 :
        print(f"Stop: trovati {len(check_deepbit_addr)} change-outputs in tx {current_transaction.iloc[0]['txId']}")
        break
    if not other_addrs.empty:
        # Add the outputs that do not belong to Deepbit.net to the set
        OthersDeepbit.update(other_addrs['addressId'].unique())
        # print(f"Found {len(OthersDeepbit)} outputs that do not belong to Deepbit.net in tx {current_transaction.iloc[0]['txId']}")

    
    # Find the next transaction
    next_transactions = inputs_info.merge(deepbit_addrs, on=['prevTxId', 'prevTxpos'])#.rename(columns={'prevTxId':'txId', 'prevTxpos':'position' })
    if next_transactions.empty:
        print(f"Stop: la catena della transazione {current_transaction['txId']} si ferma qui.")
        break
    # Add the next transaction to the graph

    next_tx_id = next_transactions.iloc[0]['txId']

    print(f"Adding transaction {next_tx_id} to the graph")
    deepbit_graph.add_node(next_tx_id)
    
    current_tx_id = current_transaction.iloc[0]['txId']
    change_address = deepbit_addrs.iloc[0]['addressId']  # change address
    
    # Add an edge from the current transaction to the next transaction
    deepbit_graph.add_edge(current_tx_id,next_tx_id, change_address=change_address)
    print(f"Added edge from {current_tx_id} to {next_tx_id} with change address {change_address}")
    # Update the current transaction to the next one
    current_transaction = next_transactions

print(f"Total number of transactions in the graph: {deepbit_graph.number_of_nodes()}")
print(f"Number of OthersDeepbit addresses: {len(OthersDeepbit)}")

In [ ]:
# Plot the graph 
posizione_circular = nx.spiral_layout(deepbit_graph, equidistant=True, resolution=1)
plt.figure(figsize=(12, 12))
nx.draw_networkx(deepbit_graph, pos= posizione_circular, node_color='lightblue', font_size=7, font_color='black', arrows=True, node_size=100)
# Highlight the start node in red
nx.draw_networkx_nodes(deepbit_graph, nodelist=[list(deepbit_graph.nodes())[0]], node_color='red', node_size=200, label='Start Node', pos=posizione_circular)

nx.draw_networkx_edge_labels(deepbit_graph, pos=posizione_circular, edge_labels={(u, v): d['change_address'] for u, v, d in deepbit_graph.edges(data=True)}, font_size=4)

# 2)

per ogni coppia di transazioni ti, ti+1 della catena calcolare rispettivamente, la differenza tra i timestamp delle due transazioni e tra i valori inviati sui change address e visualizzare le differenze ottenute su un grafico;


In [ ]:
txId_chain = pd.Series(list(deepbit_graph.nodes()), name='txId')

payment_info = transactions[['timestamp', 'txId']].merge(txId_chain, on='txId').reset_index()

# Calcola le differenze tra timestamp consecutivi
diff_timestamp = pd.Series(payment_info['timestamp'].diff().shift(-1), name='timestamp_diff')

# Calcola le differenze tra gli amount consecutivi sui change address
amount_info = outputs.merge(payment_info, on='txId')
amount_diff = pd.Series(amount_info['amount'].diff().shift(-1).abs()/ 1e8, name='amount_diff')

fig, axs = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Differenze tra timestamp e amount sui change address nella catena DeepBit.net', fontsize=16)

# Primo subplot: differenza timestamp
axs[0].plot(diff_timestamp.index, diff_timestamp, color='tab:blue', marker='o')
axs[0].set_title('Differenza timestamp tra transazioni consecutive')
axs[0].set_xlabel('Indice nella catena')
axs[0].set_ylabel('Differenza timestamp (ms)')
axs[0].grid(True, alpha=0.3)

# Secondo subplot: differenza amount
axs[1].plot(amount_diff.index, amount_diff, color='tab:orange', marker='x')
axs[1].set_title('Differenza amount tra transazioni consecutive')
axs[1].set_xlabel('Indice nella catena')
axs[1].set_ylabel('Differenza amount (satoshi)')
axs[1].grid(True, alpha=0.3)
# axs[1].ticklabel_format(style='plain', axis='y')  # Disabilita la notazione scientifica sull'asse y

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


# 3)

considerare gli indirizzi in OthersDeepbit, e verificare, effettuando scraping su WalletExplorer, se corrispondono a entità deanonimizzate;


In [ ]:
REQUEST_INTERVAL = 6  # secondi tra le richieste

def is_deanonymized(address):
    """Effettua scraping su Wallet Explorer per ottenere l'hash del wallet dato un address"""

    
    url = f"https://www.walletexplorer.com/?q={address}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    try:
        time.sleep(REQUEST_INTERVAL)  # Attendi tra le richieste per evitare blocchi
        
        response = requests.get(url, headers=headers, timeout=50)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Tentativo 1: Cerca nel div header
        main_div = soup.find('div',id='main')
        h2_tag = main_div.find('h2') if main_div else None
        
        if h2_tag:
            # Prendi il testo completo
            full = h2_tag.get_text(separator=' ').strip()
            # Estrai tutte le sottostringhe fra virgolette doppie
            quoted = full.split(' ')
            # Pulisci gli elementi e verifica che ci sia almeno un secondo elemento
            cleaned = [s.strip() for s in quoted]
            if len(cleaned) >= 2:
                wallet_hash = cleaned[2]
                if wallet_hash[0] != '[' and wallet_hash[-1] != ']':
                    # Rimuovi le parentesi quadre
                    return False

                return True
        
        # Se non trovato
        return None
        
    except Exception as e:
        print(f"Errore durante lo scraping per {address}: {str(e)}")
        return None
    

otheraddresses_mapped = mapping.merge(pd.Series(list(OthersDeepbit), name='addressId'), on='addressId')['hash']


deanonymized_addresses = set()
for address in otheraddresses_mapped:
    res = is_deanonymized(address)
    if res is not None and res:
        deanonymized_addresses.add(address)
        print(f"Address {address} is deanonymized.")

print(f"Total deanonymized addresses: {len(deanonymized_addresses)}")



## DiceOnCrack.com gambling service analysis


In [ ]:

# if diceoncrack_wallet_addresses is None:
    #in case of error in the scraping process, we can use a predefined list of wallet addresses
diceoncrack_wallet_addresses = pd.Series([
    "12TaAbLWBNKB1NLYH92CPnC1DizQoNK6FN",
    "1CRACkbiJSxfDaLNEoaNsHjNtU4KttwHyo",
    "1CRACKafkXsQzUYmu2fUM3j9c2y4yDhvfh",
    "1CRACKLiwFrZbAQz1yb9w22onHCMLbiMTY",
    "12tAabLFLxvUzC5KuX7VKMM8bYRncbQ84E",
    "1CrAcKt3HE8LNsx4KKDvjqLvcr373wg5ke",
    "1AVFypuG2jUrYzjZa69C7hK59XkWUwvK1m",
    "1CRACK25QvpVdcEmPZVD5ixtf99cMF9stg",
    "1CracksLRtQMcTF4HXNrvPzRgvz7Qr6wNd",
    "13TAabLHjNzwg8Mj7XYn76FuVAqj32s8EM",
    "1CrAckQppdcfiiw4XzpsKrZrf9eDvUok9C",
    "19TAABLQTLxgWHTdm7yNJNstgeQFgxTP4f",
    "14TAAbLiw2QLuRJCGQ3iETYg3vcpweZkTE",
    "15TaABLmhxiRQ9DTX6ZcZ9S9RknVZmP5jX",
    "1tAabLBcZLVL7md9nAnvGMCYdbvq4UVZV",
    "1PipEaL8yRS8n93mUS16wT5SNDiMrMutv5",
    "1PipemCUjxq9LKww7CaLWUMeGVZL3bD3VM",
    "1LQXotaEjfmerkwrGB3dHnheujo7sng6vA",
    "1PipeBMryPGnN3Ms3HfnNjetCS4THmkpkS",
    "1PipeZHgQXcjAYsUQ4WRXyKZn1X3sJNrpk",
    "1PipePezjvE7vBukPyDUkhHEF54qK1nkeu",
    "1Q44t4knYY3PsQZUFAejhd7Wot79ecHe8e",
    "1F4VXTQRzVQfLaGEWcf697xj1g2cKqPire",
    "1Pipeb5iNYmURifrxPZfvwHsTiw9rEb2iu",
    "1PipeZofhJv1hxsxCadEeG1vHAK87f23LE",
    "17ZmFwCULT44K25kWDeYbHiGaJCrWtytjx",
    "13encD1Yagh8M6a9Wgb3YJxKHrHqXnYi8y",
    "1GD2EiVa1rbbXcmFceyM47YN16fzVwn9j"
],name='hash')


diceoncrack_mapped_addresses = map_wallet_addresses(diceoncrack_wallet_addresses, mapping)



### 1) Find the transaction of DiceOnCrack.com which are the transactions that have at least one input or output address of DiceOnCrack.com


In [ ]:
# Find the transactions related to DiceOnCrack.com
diceoncrack_transactions = find_wallet_transactions(diceoncrack_mapped_addresses, transactions, inputs, outputs)
# Order by blockId in ascending order
diceoncrack_transactions = diceoncrack_transactions.sort_values(by='blockId', ascending=True)
print(f"Number of transactions related to DiceOnCrack.com: {len(diceoncrack_transactions)}")
# DEBUG
# print(diceoncrack_transactions.head(20))
# print(diceoncrack_transactions.describe())


### 2) Group transactions by block height


In [ ]:
def group_diceoncrack_transactions_by_block(diceoncrack_transactions):
    """Groups DiceOnCrack.com transactions by blockId."""
    
    #find the group of transactions that have the same blockId
    grouped_transactions = diceoncrack_transactions.groupby('blockId')
    return grouped_transactions


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
# Set the period : example starting from 26/12/2012
WALLET_SCRAPING_CACHE = {}
if WALLET_SCRAPING_CACHE is None or len(WALLET_SCRAPING_CACHE) == 0:
    WALLET_SCRAPING_CACHE = pd.read_csv('wallet_scraping_cache.csv', index_col=0).to_dict(orient='index')


REQUEST_INTERVAL = 7  # secondi tra le richieste

def get_wallet_hash(address):
    """Effettua scraping su Wallet Explorer per ottenere l'hash del wallet dato un address"""
    if address in WALLET_SCRAPING_CACHE:
        return WALLET_SCRAPING_CACHE[address]
    
    url = f"https://www.walletexplorer.com/?q={address}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    try:
        time.sleep(REQUEST_INTERVAL)  # Attendi tra le richieste per evitare blocchi
        
        response = requests.get(url, headers=headers, timeout=50)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        main_div = soup.find('div',id='main')
        h2_tag = main_div.find('h2') if main_div else None
        
        if h2_tag:
            # Prendi il testo completo
            full = h2_tag.get_text(separator=' ').strip()
            # Estrai tutte le sottostringhe fra virgolette doppie
            quoted = full.split(' ')
            # Pulisci gli elementi e verifica che ci sia almeno un secondo elemento
            cleaned = [s.strip() for s in quoted]
            if len(cleaned) >= 2:
                wallet_hash = cleaned[2]
                if wallet_hash[0] == '[' and wallet_hash[-1] == ']':
                    # Rimuovi le parentesi quadre
                    wallet_hash = wallet_hash[1:-1]
                # cache e rate‑limit
                # print(f"Wallet hash trovato per {address}: {wallet_hash}")
                # get_wallet_hash.last_request_time = current_time
                WALLET_SCRAPING_CACHE[address] = wallet_hash
                return wallet_hash
        
        # Se non trovato
        return None
        
    except Exception as e:
        print(f"Errore durante lo scraping per {address}: {str(e)}")
        return None
    # finally:
        # get_wallet_hash.last_request_time = time.time()

def cluster_by_wallet(group_info, diceoncrack_wallet_ids):

    grouped_infor_by_txid = group_info.groupby('txId')

    cluster_info =[]

    inputs_current_transactions= inputs.merge(group_info,on ='txId')
    renamed_outputs = outputs.rename(columns={'txId': 'prevTxId', 'position': 'prevTxpos'})
    tmp = inputs_current_transactions.merge(renamed_outputs, on=['prevTxId', 'prevTxpos'])
    

    for tx_id, group in grouped_infor_by_txid:

        inputs_mapped_address= tmp[tmp['txId'] == tx_id]['addressId']
        if inputs_mapped_address.empty:
            print(f"No inputs found for txId {tx_id}. Skipping.")
            continue

        print(f"Processing txId {tx_id} with {len(inputs_mapped_address)} input addresses.")

        # Exclude the cluster if any address belongs to DiceOnCrack
        # a =inputs_mapped_address # DEBUG
        inputs_mapped_address = inputs_mapped_address[inputs_mapped_address.isin(diceoncrack_wallet_ids) == False]
        if inputs_mapped_address.empty:
            print(f"Skipping txId {tx_id} due to no valid addresses.")
            continue

        input_address = mapping.merge(inputs_mapped_address, on='addressId')['hash']

        print("\nIndirizzi da controllare:", input_address.tolist(),"\n")
        cluster_wallet = get_wallet_hash(input_address.iloc[0])  # Prendi il wallet hash del primo address
        if len(input_address) > 1:
            for addr in input_address:
                try:
                    wallet_hash = get_wallet_hash(addr)
                    if wallet_hash != cluster_wallet:
                        print(f"Cluster skipped for txId {tx_id} due to different wallet hashes.")
                        cluster_wallet = None
                        break
                except Exception as e:
                    print(f"Error while getting wallet hash for address {addr}: {e}")
                    break
        
        if cluster_wallet is None:
            # print(f"Skipping cluster for txId {tx_id} due to inconsistent wallet hashes.")
            continue
        else:
            # print(f"Cluster for txId {tx_id} belongs to wallet: {cluster_wallet}")
            # save the cluster information
            cluster_info.append({
                'txId': tx_id,
                'wallet': cluster_wallet,
                'num_addresses': len(input_address)})

    
    # Convert the list of dictionaries to a DataFrame
    cluster_wallets = pd.DataFrame(cluster_info, columns=['txId', 'wallet','num_addresses']) if len(cluster_info) > 0 else pd.DataFrame()
    # print(cluster_wallets.info())
    # print(cluster_wallets.head(5))
    # print("------------------------------------\n")
    # print(f"Number of clusters found: {len(cluster_wallets)}")
    # print(cluster_wallets.head(20))
    return cluster_wallets


In [ ]:
from collections import defaultdict


def create_address_clusters(group_info, diceoncrack_wallet_ids, inputs_df, mapping_df):
    """
    Crea cluster di transazioni per ogni indirizzo di input
    
    Args:
        group_info: DataFrame con le transazioni di interesse
        diceoncrack_wallet_ids: Lista di addressId da escludere
        inputs_df: DataFrame degli input
        mapping_df: DataFrame di mappatura addressId -> hash
    
    Returns:
        DataFrame con cluster di transazioni per indirizzo
    """
    # Filtra solo le transazioni di interesse
    inputs_current_transactions= inputs.merge(group_info,on ='txId')
    renamed_outputs = outputs.rename(columns={'txId': 'prevTxId', 'position': 'prevTxpos'})
    relevant_inputs = inputs_current_transactions.merge(renamed_outputs, on=['prevTxId', 'prevTxpos'])
    

    
    # Unisci con la mappatura per ottenere gli hash degli indirizzi
    inputs_with_hash = relevant_inputs.merge(mapping_df,on='addressId')
    
    # Filtra gli indirizzi esclusi
    filtered_inputs = inputs_with_hash[
        ~inputs_with_hash['addressId'].isin(diceoncrack_wallet_ids)
    ]
    if filtered_inputs.empty:
        print("No valid addresses found after filtering. Returning empty DataFrame.")
        return pd.DataFrame(columns=['address', 'wallet', 'transactions', 'num_transactions'])
    
    # Dizionari per memorizzare i cluster
    address_to_transactions = defaultdict(set)
    address_to_wallet = {}
    
    # Processa ogni indirizzo univoco
    unique_addresses = filtered_inputs['hash'].unique()
    for address in unique_addresses:
        # Ottieni l'hash del wallet
        wallet_hash = get_wallet_hash(address)
        if wallet_hash:
            address_to_wallet[address] = wallet_hash
            
            # Trova tutte le transazioni per questo indirizzo
            address_txs = filtered_inputs[filtered_inputs['hash'] == address]['txId']
            address_to_transactions[address].update(address_txs)
    
    # Costruisci il risultato
    cluster_data = []
    for address, transactions in address_to_transactions.items():
        cluster_data.append({
            'address': address,
            'wallet': address_to_wallet[address],
            'transactions': transactions,
            'num_transactions': len(transactions)
        })
    
    return pd.DataFrame(cluster_data)

In [ ]:
starting_period = pd.to_datetime('2012-12-26')

diceoncrack_transactions = diceoncrack_transactions[
    diceoncrack_transactions['timestamp'] >= starting_period
]

print(f"Number of transactions after filtering by date: {len(diceoncrack_transactions)}")
# print(diceoncrack_transactions.head(5))

groups = group_diceoncrack_transactions_by_block(diceoncrack_transactions)
print(f"Number of blocks with DiceOnCrack.com transactions: {len(groups)}")
# # DEBUG
# print(groups.head(20))
# print(groups.head(5))

# Esempio di utilizzo
frames = []

for block_id, group in groups:
    tx_ids = group['txId']
    
    # # Ottimizzazione: calcola tutto in vettori
    # clusters = cluster_by_wallet(
    #     group_info=group,
    #     diceoncrack_wallet_ids=diceoncrack_mapped_addresses
    # )
    # print("Clusters\n", clusters.head(15))  # DEBUG

    # if len(clusters) > 0:
    #     clusters['blockId'] = block_id
    #     frames.append(clusters)
    clusters = create_address_clusters(
        group_info=group,
        diceoncrack_wallet_ids=diceoncrack_mapped_addresses,
        inputs_df=inputs,
        mapping_df=mapping
    )
    if clusters.empty:
        print(f"No clusters found for block {block_id}.")
        continue
    print(f"Number of transaction clusters found in block {block_id}: {len(clusters)}")
    # print(clusters.head(15))  # DEBUG
    frames.append(clusters)

# all_clusters['blockId'] = groups['blockId']

all_clusters = pd.concat(frames, ignore_index=True, axis=0) if frames else pd.DataFrame()

print(all_clusters.head(50))


In [ ]:
# all_clusters = pd.concat(frames, ignore_index=True, axis=0) if frames else pd.DataFrame()

# print(all_clusters.head(5))

# agroupby_wallet = all_clusters.groupby('wallet').agg({
#     'transactions': lambda x: set().union(*x),
#     'num_transactions': 'sum'
# }).reset_index()



In [ ]:
if not all_clusters.empty:
    groupby_block= all_clusters['blockId'].unique().sort()

    average_cluster_size = all_clusters[all_clusters['num_transactions'] >= 2].groupby('blockId')['num_transactions'].mean().reset_index()
    average_cluster_size.rename(columns={'num_transactions': 'average_size'}, inplace=True)
    print(f"Average cluster size for blocks: {average_cluster_size.head(10)}")

In [ ]:
#For DEBUG purposes
## Create a cache for the cluster wallets research
# In this case the file save the research of the clusters wallets starting from 27/12/2012
try:
    all_clusters = pd.concat(frames, ignore_index=True, axis=0) if frames else pd.DataFrame()
    all_clusters.to_csv('cluster_results_26_12_2012.csv', 
                       index=False, 
                       encoding='utf-8')  # -sig per compatibilità Excel
    
    WALLET_SCRAPING_CACHE_pd = pd.DataFrame.from_dict(WALLET_SCRAPING_CACHE, orient='index', columns=['hash']).reset_index()
    WALLET_SCRAPING_CACHE_pd.to_csv('wallet_scraping_cache.csv', 
                                    index=False, 
                                    encoding='utf-8')  # -sig per compatibilità Excel
    print("File salvato correttamente")
except Exception as e:
    print(f"Errore nel salvataggio: {e}")

#Load the clusters from the file if it exists
if all_clusters.empty or all_clusters is None:
    all_clusters = pd.read_csv('cluster_results_26_12_2012.csv', 
                               encoding='utf-8')  # -sig per compatibilità Excel


In [ ]:
# #3 Calcola la dimensione media dei cluster per ogni blocco, considerando solo cluster con almeno 2 indirizzi
# if not all_clusters.empty:
#     # Filtra prima i cluster con almeno 2 indirizzi
#     filtered = all_clusters[all_clusters['num_addresses'] >= 2]
    
#     # Calcola la media per blocco
#     avg_cluster_size_per_block = filtered.groupby('blockId')['num_addresses'].mean().reset_index()

#     if not avg_cluster_size_per_block.empty:
#         print("Dimensione media dei cluster (>=2) per blocco:")
#         # print(avg_cluster_size_per_block)
        
#         # Grafico
#         plt.figure(figsize=(12, 6))
#         avg_cluster_size_per_block.plot(kind='bar', x='blockId', y='num_addresses', legend=False)
#         plt.title('Dimensione media dei cluster (>=2) per blocco')
#         plt.xlabel('blockId')
#         plt.ylabel('Dimensione media cluster')
#         plt.tight_layout()
#         plt.show()
#     else:
#         print("Nessun cluster con almeno 2 indirizzi trovato.")
# else:
#     print("Nessun cluster disponibile per il calcolo.")

#### 4) Discussion about
